In [1]:
import os

print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 6\IEEE Access\DroMoLog\notebooks


In [2]:
import os
os.chdir("..")

In [3]:
print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 6\IEEE Access\DroMoLog


In [4]:
import json
import pandas as pd
def build_recap_multitask(model, source_dir, output_dir, setting='overall'):
    recap_overall = pd.DataFrame()
    recap_per_class = pd.DataFrame()
    dataset_dirs = os.listdir(source_dir)
    for dataset_dir in dataset_dirs:
        dataset = dataset_dir
        dataset_path = os.path.join(source_dir, dataset_dir)
        encoders = os.listdir(dataset_path)
        for encoder in encoders:
            encoder_path = os.path.join(dataset_path, encoder)
            class_weights = os.listdir(encoder_path)
            for weight in class_weights:
                weight_path = os.path.join(encoder_path, weight)
                losses = os.listdir(weight_path)
                for loss in losses:
                    loss_path = os.path.join(weight_path, loss)
                    poolings = os.listdir(loss_path)
                    for pooling in poolings:
                        pool_path = os.path.join(loss_path, pooling)
                        n_layers = os.listdir(pool_path)
                        for n_layer in n_layers:
                            layer_path = os.path.join(pool_path, n_layer)
                            n_heads = os.listdir(layer_path)
                            for n_head in n_heads:
                                head_path = os.path.join(layer_path, n_head)
                                bidirectionals = os.listdir(head_path)
                                for bidirectional in bidirectionals:
                                    bidirectional_path = os.path.join(head_path, bidirectional)
                                    files = os.listdir(bidirectional_path)
                                    eval_report_path = os.path.join(bidirectional_path, files[1])
                                    scenario_path = os.path.join(bidirectional_path, files[4])
                                    with open(eval_report_path) as eval_report_file:
                                        eval_report = json.load(eval_report_file)
                                        with open(scenario_path) as scenario_json_file:
                                            scenario = json.load(scenario_json_file)
                                            if setting == 'overall':
                                                prediction_path = os.path.join(bidirectional_path, 'prediction.xlsx')
                                                if os.path.exists(prediction_path):
                                                    prediction_df = pd.read_excel(prediction_path)
                                                else:
                                                    prediction_df = pd.read_csv(os.path.join(bidirectional_path, 'prediction.csv'))
                                                    
                                                confidence_avg = prediction_df['prob'].mean()
                                                confidence_std = prediction_df['prob'].std()
                                                accuracy = float(round(eval_report['accuracy'] * 100, 5))
                                                precision = float(round(eval_report['weighted_avg']['precision'] * 100, 5))
                                                recall = float(round(eval_report['weighted_avg']['recall'] * 100, 5))
                                                f1_score = float(round(eval_report['weighted_avg']['f1-score'] * 100, 5))
                                                best_epoch = int(scenario['best_epoch'])
                                                columns = ['dataset', 'encoder', 'n_layer', 'n_head', 'bidirectional', 'pooling', 'class_weight', 'scenario', 'best_epoch', 'accuracy', 'precision', 'recall', 'f1_score', 'confidence_avg', 'confidence_std']
                                                row_list = [dataset, encoder, n_layer, n_head, bidirectional, pooling, weight, loss, best_epoch, accuracy, precision, recall, f1_score, confidence_avg, confidence_std]
                                                scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                                                recap_overall = pd.concat([recap_overall, scenario_df], ignore_index=True)
                                                if not os.path.exists(output_dir):
                                                    os.makedirs(output_dir)
                                                recap_overall.to_excel(os.path.join(output_dir, f'recap_overall_{model}.xlsx'), index=False, float_format='%.5f')
                                            else:
                                                accuracy = float(round(eval_report['accuracy'] * 100, 5))
                                                best_epoch = int(scenario['best_epoch'])
                                                high_score = [float(round(eval_report['high'][key] * 100, 5)) for key in eval_report['high'].keys()]
                                                low_score = [float(round(eval_report['low'][key] * 100, 5)) for key in eval_report['low'].keys()]
                                                medium_score = [float(round(eval_report['medium'][key] * 100, 5)) for key in eval_report['medium'].keys()]
                                                normal_score = [float(round(eval_report['normal'][key] * 100, 5)) for key in eval_report['normal'].keys()]
                                                columns = ['dataset', 'encoder', 'n_layer', 'n_head', 'bidirectional', 'pooling', 'class_weight', 'scenario', 'best_epoch', 'accuracy', 'high_precision', 'high_recall', 'high_f1_score', 'medium_precision', 'medium_recall', 'medium_f1_score', 'low_precision', 'low_recall', 'low_f1_score', 'normal_precision', 'normal_recall', 'normal_f1_score']
                                                row_list = [dataset, encoder, n_layer, n_head, bidirectional, pooling, weight, loss, best_epoch, accuracy, high_score[0], high_score[1], high_score[2], medium_score[0], medium_score[1], medium_score[2], low_score[0], low_score[1], low_score[2], normal_score[0], normal_score[1], normal_score[2]]
                                                scenario_df = pd.DataFrame([row_list], index=None, columns=columns)
                                                recap_per_class = pd.concat([recap_per_class, scenario_df], ignore_index=True)
                                                recap_per_class.to_excel(os.path.join(output_dir, f'recap_per_class_{model}.xlsx'), index=False, float_format='%.5f')

#### Our Own Baseline

In [ ]:
model = 'multitask_15'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'baseline'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')

#### State-of-the-art Baseline

In [6]:
model = 'transsentlog_15'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'transsentlog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')

In [ ]:
model = 'neurallog_15'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'overall')

In [ ]:
model = 'neurallog'
source_dir = os.path.join('experiments', model)
output_dir = os.path.join('experiments', 'recap', model)
build_recap_multitask(model, source_dir, output_dir, 'per_class')